In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

#set the max columns to none
pd.set_option('display.max_columns', None)

In [19]:
size = 'demo'
#Import train Data
history_train_path = f'./files/parquet/ebnerd_{size}/train/history.parquet'
history_train_df = pd.read_parquet(history_train_path)
print(f'Raw {size} train history df shape:          ',history_train_df.shape)

behaviors_train_path = f'./files/parquet/ebnerd_{size}/validation/behaviors.parquet'
behaviors_train_df = pd.read_parquet(behaviors_train_path)
print(f'Raw {size} train behaviors df shape:          ',behaviors_train_df.shape)

# articles_train_path = f'./files/parquet/ebnerd_{size}/articles.parquet'
# articles_df = pd.read_parquet(articles_train_path)
# print(f'Raw {size} articles df shape:          ',articles_df.shape)

# Ready embeddings
embeddings_path = f'./files/parquet/Ekstra_Bladet_word2vec/document_vector.parquet'
embeddings_df = pd.read_parquet(embeddings_path).tail(60000)
print(f'embeddings df shape:          ',embeddings_df.shape)

Raw demo train history df shape:           (1590, 5)
Raw demo train behaviors df shape:           (25356, 17)
embeddings df shape:           (60000, 2)


In [3]:
print(history_train_df.explode('impression_time_fixed').impression_time_fixed.min())
print(history_train_df.explode('impression_time_fixed').impression_time_fixed.max())


print(behaviors_train_df.impression_time.min())
print(behaviors_train_df.impression_time.max())

2023-04-27 07:00:05
2023-05-18 06:59:51
2023-05-25 07:00:15
2023-06-01 06:59:33


In [4]:
embeddings_df.head()

,article_id,document_vector
0,3000022,"[0.06542388, -0.047424573, 0.06384871, -0.0014..."
1,3000063,"[0.028815078, -0.00016637295, 0.055056807, 0.0..."
2,3000613,"[0.037971217, 0.03392251, 0.027297212, 0.01708..."
3,3000700,"[0.04652399, 0.0029133065, 0.06280604, -0.0051..."
4,3000840,"[0.014736942, 0.024067875, 0.0051865038, 0.041..."


In [5]:
history_train_df.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,13538,"[2023-04-27T10:17:43.000000, 2023-04-27T10:18:...","[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11..."
1,58608,"[2023-04-27T18:48:09.000000, 2023-04-27T18:48:...","[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,..."
2,95507,"[2023-04-27T15:20:28.000000, 2023-04-27T15:20:...","[60.0, 100.0, 100.0, 21.0, 29.0, 67.0, 49.0, 5...","[9739035, 9738646, 9634967, 9738902, 9735495, ...","[18.0, 29.0, 51.0, 12.0, 10.0, 10.0, 13.0, 24...."
3,106588,"[2023-04-27T08:29:09.000000, 2023-04-27T08:29:...","[24.0, 57.0, 100.0, nan, nan, 100.0, 100.0, 73...","[9738292, 9738216, 9737266, 9737556, 9737657, ...","[9.0, 15.0, 42.0, 9.0, 3.0, 58.0, 26.0, 214.0,..."
4,617963,"[2023-04-27T14:42:25.000000, 2023-04-27T14:43:...","[100.0, 100.0, nan, 46.0, 23.0, 19.0, 61.0, 70...","[9739035, 9739088, 9738902, 9738968, 9738760, ...","[45.0, 29.0, 116.0, 26.0, 34.0, 42.0, 58.0, 59..."


In [8]:
articles_df.head()

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3037230,Ishockey-spiller: Jeg troede jeg skulle dø,ISHOCKEY: Ishockey-spilleren Sebastian Harts h...,2023-06-29 06:20:57,False,Ambitionerne om at komme til USA og spille ish...,2003-08-28 08:55:00,None,article_default,https://ekstrabladet.dk/sport/anden_sport/isho...,[],[],"[Kriminalitet, Kendt, Sport, Katastrofe, Mindr...",142,"[327, 334]",sport,NaN,NaN,NaN,0.9752,Negative
1,3044020,Prins Harry tvunget til dna-test,Hoffet tvang Prins Harry til at tage dna-test ...,2023-06-29 06:21:16,False,Den britiske tabloidavis The Sun fortsætter me...,2005-06-29 08:47:00,"[3097307, 3097197, 3104927]",article_default,https://ekstrabladet.dk/underholdning/udlandke...,"[Harry, James Hewitt]","[PER, PER]","[Kriminalitet, Kendt, Underholdning, Personfar...",414,[432],underholdning,NaN,NaN,NaN,0.7084,Negative
2,3057622,Rådden kørsel på blå plader,Kan ikke straffes: Udenlandske diplomater i Da...,2023-06-29 06:21:24,False,Slingrende spritkørsel. Grove overtrædelser af...,2005-10-10 07:20:00,[3047102],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,[],[],"[Kriminalitet, Transportmiddel, Bil]",118,[133],nyheder,NaN,NaN,NaN,0.9236,Negative
3,3073151,Mærsk-arvinger i livsfare,FANGET I FLODBØLGEN: Skibsrederens oldebørn må...,2023-06-29 06:21:38,False,To oldebørn af skibsreder Mærsk McKinney Mølle...,2005-01-04 06:59:00,"[3067474, 3067478, 3153705]",article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,[],[],"[Erhverv, Privat virksomhed, Livsstil, Familie...",118,[133],nyheder,NaN,NaN,NaN,0.9945,Negative
4,3193383,Skød svigersøn gennem babydyne,44-årig kvinde tiltalt for drab på ekssvigersø...,2023-06-29 06:22:57,False,En 44-årig mormor blev i dag fremstillet i et ...,2003-09-15 15:30:00,None,article_default,https://ekstrabladet.dk/krimi/article3193383.ece,[],[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9966,Negative


In [11]:
articles_lst = articles_df['article_id'].tolist()

In [20]:
embeddings_lst = embeddings_df['article_id'].tolist()

In [21]:
history_articles_lst = list(set(history_train_df['article_id_fixed'].explode().tolist()))

In [22]:
behaviors_inview_articles_lst = list(set(behaviors_train_df['article_ids_inview'].explode().tolist()))

In [23]:
behaviors_clicked_articles_lst = list(set(behaviors_train_df['article_ids_clicked'].explode().tolist()))

In [24]:
# print(len(articles_lst))
print(len(embeddings_lst))
print(len(history_articles_lst))
print(len(behaviors_inview_articles_lst))
print(len(behaviors_clicked_articles_lst))

60000
4247
2738
1144


In [25]:
final_lst =  []
final_lst.extend(history_articles_lst)
final_lst.extend(behaviors_inview_articles_lst)
final_lst.extend(behaviors_clicked_articles_lst)

final_lst = list(set(final_lst))

In [26]:
len(final_lst)

6204

In [27]:
counter =0
for art in tqdm(final_lst):
    if art in embeddings_lst:
        counter += 1

/var/folders/ld/c07vs56s18bc0h2krlvl0myr0000gn/T/ipykernel_3210/1777265541.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for art in tqdm(final_lst):


  0%|          | 0/6204 [00:00<?, ?it/s]

In [28]:
print(counter)

5900


In [31]:
history_train_df['user_id'].unique().shape

(1590,)

In [32]:
behaviors_train_df.head()

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,144772,NaN,2023-05-30 14:21:34,29.0,NaN,2,"[9788239, 9780702, 9553264, 9787499, 6741781, ...",[9783042],76658,False,NaN,NaN,NaN,False,29,7.0,59.0
1,144777,NaN,2023-05-30 14:22:11,10.0,NaN,2,"[9788521, 9786217, 9553264, 9788361, 9788352, ...",[9788125],76658,False,NaN,NaN,NaN,False,29,58.0,98.0
2,196487,NaN,2023-05-27 19:54:18,16.0,NaN,2,"[9279095, 9784273, 9784275, 9784506, 9784444, ...",[9782806],760446,False,NaN,NaN,NaN,False,220,64.0,100.0
3,196495,NaN,2023-05-27 19:53:48,25.0,NaN,2,"[9784575, 9784607, 9784559, 9784662, 9783852, ...",[9782656],760446,False,NaN,NaN,NaN,False,220,4.0,56.0
4,196496,NaN,2023-05-27 19:56:28,11.0,NaN,2,"[9784137, 9784298, 9779370, 9782517, 9777324, ...",[9777324],760446,False,NaN,NaN,NaN,False,220,65.0,87.0


In [33]:
# ----- NOT SURE IF THESE TRANSFORMATION ARE NEEDED NEITHER IF WE MAKE THE RECOMMENDATIONS IN THIS DATASET --------
behaviors_train_df = behaviors_train_df[['user_id','article_ids_inview', 'article_ids_clicked']]
behaviors_train_df = behaviors_train_df.explode('article_ids_clicked')

behaviors_train_grouped_clicked = df = behaviors_train_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index()

behaviors_train_df = behaviors_train_df.explode('article_ids_inview')

behaviors_train_grouped_inview_df = behaviors_train_df.groupby('user_id')['article_ids_inview'].apply(list).reset_index()

behaviors_train_df = pd.merge(behaviors_train_grouped_inview_df, behaviors_train_grouped_clicked, on='user_id', how='inner')# ----- NOT SURE IF THESE TRANSFORMATION ARE NEEDED NEITHER IF WE MAKE THE RECOMMENDATIONS IN THIS DATASET --------


In [34]:
behaviors_train_df.shape

(1562, 3)

In [35]:
behaviors_train_df.head()

,user_id,article_ids_inview,article_ids_clicked
0,19181,"[9783019, 9778732, 9783122, 9783024, 9783159, ...","[9783019, 9770145, 9782519, 9782899, 9788947, ..."
1,21271,"[9784097, 7594265, 9782407, 9785267, 9784852, ...","[9785113, 9339920, 9786222, 9782407, 9784097, ..."
2,21774,"[9780815, 9786111, 9777339, 9052390, 9785500, ...",[9785835]
3,22779,"[9787465, 9787261, 9787332, 9787469, 9786495, ...","[9787261, 9784852, 9780702, 9785111, 9782806, ..."
4,22895,"[9782652, 9783024, 9782202, 9783122, 9782996, ...","[9782996, 9790574, 9790475, 9781013, 9780514, ..."


In [36]:
train_df = pd.merge(history_train_df, behaviors_train_df, on='user_id', how='inner')
train_df.shape

(1217, 7)

In [37]:
train_df.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_inview,article_ids_clicked
0,95507,"[2023-04-27T15:20:28.000000, 2023-04-27T15:20:...","[60.0, 100.0, 100.0, 21.0, 29.0, 67.0, 49.0, 5...","[9739035, 9738646, 9634967, 9738902, 9735495, ...","[18.0, 29.0, 51.0, 12.0, 10.0, 10.0, 13.0, 24....","[9785868, 9210441, 9785019, 9785835, 9785899, ...","[9783057, 9769155, 9784710, 9781763, 9782046, ..."
1,106588,"[2023-04-27T08:29:09.000000, 2023-04-27T08:29:...","[24.0, 57.0, 100.0, nan, nan, 100.0, 100.0, 73...","[9738292, 9738216, 9737266, 9737556, 9737657, ...","[9.0, 15.0, 42.0, 9.0, 3.0, 58.0, 26.0, 214.0,...","[9780815, 9783803, 9785596, 9786468, 9786311, ...","[9776918, 9786268, 9779370, 9783952, 9780348, ..."
2,750497,"[2023-04-28T22:12:20.000000, 2023-04-28T22:18:...","[100.0, 100.0, 100.0, 100.0, 100.0, 71.0, 67.0...","[9741428, 9741259, 9741428, 9739597, 9741428, ...","[380.0, 172.0, 189.0, 167.0, 85.0, 0.0, 25.0, ...","[9789125, 9788524, 9788841, 9788471, 9789065, ...","[9789065, 9782845, 9780702, 9788106, 9788921, ..."
3,119480,"[2023-04-29T22:03:15.000000, 2023-04-29T22:03:...","[17.0, 11.0, 100.0, 19.0, 40.0, 22.0, 18.0, 19...","[9742440, 9742379, 9748678, 9748582, 9748807, ...","[9.0, 18.0, 53.0, 9.0, 3.0, 8.0, 4.0, 5.0, 8.0...","[9782438, 9782418, 9782616, 9782391, 9782519, ...","[9782391, 9782361, 9790752, 9781856]"
4,160892,"[2023-04-27T09:10:33.000000, 2023-04-27T09:20:...","[100.0, 20.0, 100.0, 100.0, 100.0, 15.0, 58.0,...","[9738557, 9738211, 9736646, 9739342, 9739443, ...","[583.0, 257.0, 10.0, 396.0, 28.0, 6.0, 3.0, 10...","[9695098, 9782672, 9782308, 9782519, 9782487, ...","[9782438, 9780968, 9781814, 9781859, 9781856, ..."


In [38]:
# Read cosine similarity matrix pickle file
size = 'demo'
#Import train Data
train_path = f'./files/parquet/ebnerd_{size}/train/train.pickle'

train_df.to_pickle(train_path)

In [41]:
size = 'demo'
#Import train Data
train_path = f'./files/parquet/ebnerd_{size}/train/train.pickle'
train_df = pd.read_pickle(train_path)
print(train_df.shape)

(1217, 7)


In [42]:
train_df['article_id_fixed_set_len'] = train_df['article_id_fixed'].apply(lambda lst: len(list(set(lst))))
train_df['article_ids_inview_set_len'] = train_df['article_ids_inview'].apply(lambda lst: len(list(set(lst))))
#train_df['article_ids_clicked_len'] = train_df['article_ids_clicked'].apply(lambda lst: len(lst))

In [43]:
train_df.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_inview,article_ids_clicked,article_id_fixed_set_len,article_ids_inview_set_len
0,95507,"[2023-04-27T15:20:28.000000, 2023-04-27T15:20:...","[60.0, 100.0, 100.0, 21.0, 29.0, 67.0, 49.0, 5...","[9739035, 9738646, 9634967, 9738902, 9735495, ...","[18.0, 29.0, 51.0, 12.0, 10.0, 10.0, 13.0, 24....","[9785868, 9210441, 9785019, 9785835, 9785899, ...","[9783057, 9769155, 9784710, 9781763, 9782046, ...",342,58
1,106588,"[2023-04-27T08:29:09.000000, 2023-04-27T08:29:...","[24.0, 57.0, 100.0, nan, nan, 100.0, 100.0, 73...","[9738292, 9738216, 9737266, 9737556, 9737657, ...","[9.0, 15.0, 42.0, 9.0, 3.0, 58.0, 26.0, 214.0,...","[9780815, 9783803, 9785596, 9786468, 9786311, ...","[9776918, 9786268, 9779370, 9783952, 9780348, ...",141,104
2,750497,"[2023-04-28T22:12:20.000000, 2023-04-28T22:18:...","[100.0, 100.0, 100.0, 100.0, 100.0, 71.0, 67.0...","[9741428, 9741259, 9741428, 9739597, 9741428, ...","[380.0, 172.0, 189.0, 167.0, 85.0, 0.0, 25.0, ...","[9789125, 9788524, 9788841, 9788471, 9789065, ...","[9789065, 9782845, 9780702, 9788106, 9788921, ...",294,598
3,119480,"[2023-04-29T22:03:15.000000, 2023-04-29T22:03:...","[17.0, 11.0, 100.0, 19.0, 40.0, 22.0, 18.0, 19...","[9742440, 9742379, 9748678, 9748582, 9748807, ...","[9.0, 18.0, 53.0, 9.0, 3.0, 8.0, 4.0, 5.0, 8.0...","[9782438, 9782418, 9782616, 9782391, 9782519, ...","[9782391, 9782361, 9790752, 9781856]",25,37
4,160892,"[2023-04-27T09:10:33.000000, 2023-04-27T09:20:...","[100.0, 20.0, 100.0, 100.0, 100.0, 15.0, 58.0,...","[9738557, 9738211, 9736646, 9739342, 9739443, ...","[583.0, 257.0, 10.0, 396.0, 28.0, 6.0, 3.0, 10...","[9695098, 9782672, 9782308, 9782519, 9782487, ...","[9782438, 9780968, 9781814, 9781859, 9781856, ...",265,260
